# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# Clone `CreateDebateScraper` library from github
!git clone https://github.com/utkarsh512/CreateDebateScraper.git
%cd CreateDebateScraper/src/nested/

In [ ]:
!pip install transformers

In [ ]:
from   copy                     import deepcopy
# import cpnet
from   itertools                import accumulate
import json
from   matplotlib               import pyplot as plt
import networkx as nx
import nltk
import numpy as np
import pandas as pd
import pickle
import random
import re
from   scipy                    import stats
import textwrap
from   thread                   import Comment, Thread
from   tqdm                     import tqdm
nltk.download('punkt') # For tokenizers
nltk.download('stopwords')
import matplotlib
from   nltk.tokenize            import TweetTokenizer
from   nltk.corpus              import stopwords
from   pprint                   import pprint
from   transformers             import BertModel, BertTokenizer
# import shifterator as sh
# import wordcloud
# import skbio
matplotlib.rcParams.update({'font.size': 18})
matplotlib.rcParams["figure.figsize"] = (12, 5)
STOP_WORDS = list(stopwords.words('english'))

In [ ]:
# Custom routine to clean texts scraped from Web.
# It removes hyperlinks, punctuation marks (except apostrophe)

tknz = TweetTokenizer()

def clean_text(text):
    """
    Preprocessing text
    """
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"www\S+", "", text)
    text = re.sub("-", " ", text)
    text = re.sub("\s+", " ", text)
    text = re.sub("\u2018", "X", text) 
    text = re.sub("\u2019", "X", text) 
    text = re.sub("\'", "X", text) 
    wordTokens_ = tknz.tokenize(text)
    wordTokens = list()
    for x in wordTokens_:
        x = ''.join([v for v in x if v.isalnum() or v == ' '])
        if len(x) > 0 and x != 'X':
            x = x.replace('X', '\'')
            wordTokens.append(x)
    return wordTokens

# Pre-processing CreateDeate dataset

In [ ]:
comments = dict()

# Topical forums on CreateDebate. We have scraped comments for all of the
# following forurm.
categories = ['business', 'comedy', 'entertainment', 'health', 'law', 'nsfw',
              'politics2', 'religion', 'science', 'shopping', 'sports',
              'technology', 'travel', 'world']

# However, we will be analyzing comments from selected forum only!
# These forum have at least 10k comments each.
categories_selected = ['politics2', 'religion', 'world', 
                       'science', 'law', 'technology']

for x in categories_selected:
    comments[x] = list()

In [ ]:
# Loading comments from select forums

for cat in tqdm(categories_selected):
    fp = open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/threads.log', 'rb')

    # Get all the `Thread` objects pickled while scraping.
    threads = list()
    try:
        while True:
            e = pickle.load(fp)
            threads.append(e)
    except EOFError:
        fp.close()

    # While classifying CreateDebate comments, we used comments as per author mode.
    # Hence, using the same mode to attach classification score with the comments.
    # 
    # score < 0.5 -> ad hominem comment
    #       > 0.5 -> non ad hominem comment
    authors = dict()
    for thread in threads:
        for k, v in thread.comments.items():
            try:
                authors[v.author].append((v, k))
            except:
                authors[v.author] = list()
                authors[v.author].append((v, k))

    ctr = 0
    # Load the classification score of the comments.
    with open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/comments_with_score.log', 'rb') as fp:
        cws = pickle.load(fp)
    # Attach classification score with the comments.
    for author in authors.keys():
        for i in range(len(authors[author])):
            comment, cid = authors[author][i]
            foo = deepcopy(comment.__dict__)
            foo['tag'] = cat
            foo['score'] = cws[ctr][0]
            foo['validation'] = cws[ctr][1][0]
            foo['id'] = int(cid[3:])
            comments[cat].append(foo)
            ctr += 1

In [ ]:
ah_score_comments = dict()

for cat in categories_selected:
    for comment in comments[cat]:
        ah_score_comments[comment['id']] = 1 - comment['score']

In [ ]:
def parse_tstring(tstring):
    """
    Parses comment's time to an integer to enable
    comparison between comments based on their time of posting
    """
    if tstring == 'Not Available':
        raise ValueError('Invalid posting time for parse_tstring')
    tstring = tstring.replace('T', '-').replace(':', '-').replace('+', '-').split('-')
    return int(''.join(tstring[:-2]))

In [ ]:
# Loading AH score

with open('/content/gdrive/MyDrive/Temp/47-ah-score.pkl', 'rb') as fp:
    ah_score = pickle.load(fp)

# `ah_score` is a dictionary that contains the ah score of the comments written
# by all the users

# key: category -> user
# value: list of ah_score for given user for given category

# value > 0.5 --> ad hominem
# value < 0.5 --> non ad hominem

In [ ]:
# Maximum ah score per category per author
#   key: category -> author
#   value: maximum ah score

ah_score_max = dict()

for category, author_data in ah_score.items():
    ah_score_max[category] = dict()
    for author, ah_scores in author_data.items():
        ah_score_max[category][author] = np.max(ah_scores)

In [ ]:
comment_count = dict()
# key: category -> author
# value: number of comments written by author in the given forum

for category in categories_selected:
    comment_count[category] = dict()

    for comment in comments[category]:
        author = comment['author']
        try:
            comment_count[category][author] += 1
        except KeyError:
            comment_count[category][author] = 1

In [ ]:
user_list = set()

for category in categories_selected:
    for comment in comments[category]:
        user_list.add(comment['author'])

user_list = list(user_list)

In [ ]:
first_post_time = dict()
# key: category -> user
# value: post time of the first comment by given user in the given category
#        It is an integer as returned by parse_tstring routine

for category in categories_selected:
    first_post_time[category] = dict()

    for comment in comments[category]: 
        if comment['time'] == 'Not Available':
            continue
        author = comment['author']
        try:
            first_post_time[category][author] = min(first_post_time[category][author], parse_tstring(comment['time']))
        except KeyError:
            first_post_time[category][author] = parse_tstring(comment['time'])

In [ ]:
def get_migrated_users(categories_1, categories_2, categories_1_origin=True, require_migration=True):
    """
    Returns a list of usernames who migrated from categories_1 to categories_2

    If categories_1_origin is True, we will consider all other major categories
    to compute post_time_2, so as to ensure that first post by the user is in 
    categories_1

    If require_migration is True, post_time_1 < post_time_2 condition is relaxed
    """

    resultant_list = list()

    for user in user_list:
        post_time_1 = 20220101000000
        post_time_2 = 20220101000000

        if not isinstance(categories_1, set):
            categories_1 = set(categories_1)
        if not isinstance(categories_2, set): 
            categories_2 = set(categories_2)
        
        for category in categories_1:
            try:
                cur_post_time = first_post_time[category][user]
                post_time_1 = min(post_time_1, cur_post_time)
            except KeyError:
                pass
        
        for category in categories_2:
            try:
                cur_post_time = first_post_time[category][user]
                post_time_2 = min(post_time_2, cur_post_time) 
            except KeyError:
                pass

        if post_time_1 == 20220101000000 or post_time_2 == 20220101000000:
            continue

        if categories_1_origin:
            for category in categories_selected:
                if not ((category in categories_1) or (category in categories_2)):
                    try:
                        cur_post_time = first_post_time[category][user]
                        post_time_2 = min(post_time_2, cur_post_time)
                    except KeyError:
                        pass

        if post_time_1 < post_time_2 or not require_migration:
            resultant_list.append(user)
        
    return resultant_list

In [ ]:
def partition_migrated_users(migration_list, categories_1, categories_2):
    """
    Partitions the users into 4 categories: 
        AH-AH
        AH-NonAH
        NonAH-AH
        NonAH-NonAH

    Users are classified as AH in a given category if they post at least one 
    ad hominem comment in that category
    
    Note: migration_list should be obtained using get_migrated_users method
    """

    ah_ah_list = []
    ah_nonah_list = []
    nonah_ah_list = []
    nonah_nonah_list = []

    for user in migration_list:
        max_score_1 = 0
        max_score_2 = 0
        for category in categories_1:
            max_score_1 = max(max_score_1, ah_score_max[category].get(user, 0))
        for category in categories_2:
            max_score_2 = max(max_score_2, ah_score_max[category].get(user, 0))

        if max_score_1 > 0.5 and max_score_2 > 0.5:
            ah_ah_list.append(user)

        elif max_score_1 > 0.5 and max_score_2 < 0.5:
            ah_nonah_list.append(user)
        
        elif max_score_1 < 0.5 and max_score_2 > 0.5:
            nonah_ah_list.append(user)

        elif max_score_1 < 0.5 and max_score_2 < 0.5:
            nonah_nonah_list.append(user)
        
        else:
            print(user)

    return ah_ah_list, ah_nonah_list, nonah_ah_list, nonah_nonah_list 

# Attention Visualizer

In [ ]:
class Visualizer:
    """Wrapper for creating heatmaps for documents"""
    def __init__(self):
        self._header = r'''\documentclass[10pt,a4paper]{article}
\usepackage[left=1.00cm, right=1.00cm, top=1.00cm, bottom=2.00cm]{geometry}
\usepackage{color}
\usepackage{tcolorbox}
\usepackage{CJK}
\usepackage{adjustbox}
\tcbset{width=0.9\textwidth,boxrule=0pt,colback=red,arc=0pt,auto outer arc,left=0pt,right=0pt,boxsep=5pt}
\begin{document}
\begin{CJK*}{UTF8}{gbsn}''' + '\n\n'

        self._footer = r'''\end{CJK*}
\end{document}'''

    def visualize(self,
                  word_list,
                  attention_list,
                  label_list,
                  latex_file,
                  title,
                  batch_size=20,
                  color='blue'):
        """Routine to generate attention heatmaps for given texts
        ---------------------------------------------------------
        Input:
        :param word_list: list of texts (each text is a list of words)
        :param attention_list: scores for each word, dimension same as word_list
        :param label_list: label for each text
        :param latex_file: name of the latex file
        :param title: title of latex file
        :param batch_size: Number of comments in each batch
        :param color: color used for visualization, can be 'blue', 'red', 'green', etc.
        """
        word_list_processed = []
        for x in word_list:
            word_list_processed.append(self._clean_word(x))

        with open(latex_file, 'w', encoding='utf-8') as f:
            f.write(self._header)
            f.write('\\section{%s}\n\n' % title)

            n_examples = len(word_list)
            n_batches = n_examples // batch_size

            for i in range(n_batches):
                batch_word_list = word_list_processed[i * batch_size: (i + 1) * batch_size]
                batch_attention_list = attention_list[i * batch_size: (i + 1) * batch_size]
                batch_label_list = label_list[i * batch_size: (i + 1) * batch_size]
                f.write('\\subsection{Batch %d}\n\n' % (i + 1))
                for j in range(batch_size):
                    f.write('\\subsubsection{Comment %d - %s}\n\n' % (j + 1, batch_label_list[j]))
                    sentence = batch_word_list[j]
                    score = batch_attention_list[j]
                    assert len(sentence) == len(score)
                    f.write('\\noindent')
                    for k in range(len(sentence)):
                        f.write('\\colorbox{%s!%s}{' % (color, score[k]) + '\\strut ' + sentence[k] + '} ')
                    f.write('\n\n')

            f.write(self._footer)

    @staticmethod
    def _clean_word(word_list):
        new_word_list = []
        for word in word_list:
            for latex_sensitive in ["\\", "%", "&", "^", "#", "_", "{", "}"]:
                if latex_sensitive in word:
                    word = word.replace(latex_sensitive, '\\' + latex_sensitive)
            new_word_list.append(word)
        return new_word_list

viz = Visualizer()

In [ ]:
model_version = '/content/gdrive/MyDrive/DL/cnerg-bert-adhominem'
do_lower_case = True
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)

In [ ]:
INTENSITY = 70

def attention_scores(text, layers=None, heads=None):
    sentence_a = text
    inputs = tokenizer.encode_plus(sentence_a, None, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    attention = model(input_ids)[-1]
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list) 
    sz = len(tokens)
    matrix = [0 for j in range(sz)]
    if layers is None:
        layers = [x for x in range(12)]
    if heads is None:
        heads = [x for x in range(12)]
    for layer in layers:
        for head in heads:
            for j in range(sz):
                matrix[j] += attention[layer][0, head, 0, j].item()
    for j in range(sz):
        matrix[j] = (matrix[j]) / (len(layers) * len(heads))
    return (tokens, matrix)

In [ ]:
def clean_array(w, a):
    W = []
    A = []
    for i in range(len(w)):
        if (w[i].startswith('##')):
            W[len(W) - 1] += w[i][2:]
            A[len(A) - 1] = (A[len(A) - 1] + a[i]) / 2
        else:
            W.append(w[i])
            A.append(a[i])
    return clean_apos(W, A)

def clean_apos(w, a):
    W = []
    A = []
    ctr = 0
    while ctr != len(w):
        if w[ctr] == '\'':
            W[-1] += w[ctr] + w[ctr + 1]
            A[-1] = min(INTENSITY, A[-1] + a[ctr] + a[ctr + 1])
            ctr += 2
        else:
            W.append(w[ctr])
            A.append(a[ctr])
            ctr += 1
    return W, A

In [ ]:
def top_three_tokens(text):
    words, attentions = attention_scores(text)
    words = words[1:-1] # Remove start and end tags
    attentions = attentions[1:-1]
    assert len(words) == len(attentions)
    words, attentions = clean_array(words, attentions)
    assert len(words) == len(attentions)
    top_tokens = list()
    for i in range(len(words)):
        top_tokens.append((attentions[i], i))
    top_tokens = sorted(top_tokens, reverse=True)
    ind = [0]
    cur = 1
    while len(ind) < 3:
        take = True
        for ids in ind:
            take = take and abs(top_tokens[ids][1] - top_tokens[cur][1]) > 2
        if take:
            ind.append(cur)
        cur += 1
    xx = []
    for x in ind:
        xx.append(top_tokens[x][1])
    scores = [0 for i in range(len(words))]
    for w in xx:
        lst = [w - 1, w, w + 1]
        for j in lst:
            if j >= 0 and j < len(words):
                scores[j] = INTENSITY
    return words, scores

In [ ]:
def create_latex_file(texts):
    words_list = list()
    scores_list = list()

    for comment in texts:
        try:
            words, scores = top_three_tokens(comment)
        except:
            continue
        words_list.append(words)
        scores_list.append(scores)
    
    label = ' '
    labels_list = [label for _ in range(len(words_list))]
    
    viz.visualize(words_list, scores_list, labels_list,
                  latex_file='sample.tex',
                  title=f'Top three triggers',
                  batch_size=len(words_list),
                  color='cyan')

# Slur Words

In [ ]:
SLUR_WORDS = {
  "jews": [
    "jews",
    "oven dodger",
    "nazi",
    "dirty jew",
    "holocaust",
    "kikesucker",
    "hook nose",
    "kike"
  ],
  "homosexual": [
    "faggots usually",
    "fucking queer",
    "the biggest faggot",
    "dyke",
    "you fucking faggot",
    "hate faggots",
    "queer",
    "homosexual",
    "the faggots",
    "faggot",
    "faggots usually have",
    "gay",
    "faggots",
    "dykey",
    "ugly dyke",
    "faggots like you",
    "you a fag",
    "lesbian",
    "homo",
    "is a faggot",
    "like a faggot",
    "dykes",
    "faggots like",
    "faggot if you ever"
  ],
  "women": [
    "ugly dyke",
    "woman terrorist",
    "nigress",
    "bitch",
    "slut",
    "women",
    "sheeboon",
    "negress",
    "mud shark",
    "women threat",
    "you a lame bitch",
    "your a cunt",
    "white bitch",
    "niggeress",
    "hoe",
    "dykes",
    "niggress",
    "sheboon",
    "feminazi"
  ],
  "blacks": [
    "pavement ape",
    "the niggers",
    "negress",
    "porch monkey",
    "that nigger",
    "this nigger",
    "sheboon",
    "all niggers",
    "eurafrica",
    "shut up nigger",
    "picaninny",
    "african attack",
    "spearchucker",
    "how many niggers",
    "nigger",
    "africa",
    "niggers are in my",
    "dindu nuffin",
    "stupid nigger",
    "moolie",
    "niggers",
    "bluegum",
    "nigger ass",
    "you niggers",
    "fucking nigger",
    "nigger music",
    "niggress",
    "you a nigger",
    "many niggers are",
    "nigress",
    "blacks",
    "teenaper",
    "sheeboon",
    "dumb nigger",
    "niggeress",
    "pickaninny",
    "nigga"
  ],
  "muslim": [
    "muslim immigrant",
    "islam",
    "mudslime",
    "mooslem",
    "muslim refugee",
    "musslime",
    "shitlam",
    "muslim invasion",
    "moslime",
    "mooslamic",
    "muzzie",
    "allah akbar",
    "mooslime",
    "musloid",
    "mudslimes",
    "muslim",
    "muslimes",
    "moslum",
    "mussie",
    "muzrat",
    "muslim countries",
    "muzzy",
    "moslim",
    "jihadi",
    "muslim country",
    "moslem",
    "muzzrat",
    "mooslim"
  ],
  "arabs": [
    "towel head",
    "goatfucker",
    "arabs",
    "goathumper",
    "raghead",
    "rag head",
    "goathumping",
    "towelhead",
    "camel jockey",
    "sandnigger",
    "camel fucker",
    "sand nigger"
  ],
  "generic": [
    "to rape",
    "raped and",
    "shithole country",
    "get raped",
    "raped",
    "is a fucking",
    "shit skin",
    "raped by",
    "hate you",
    "fake empowerment",
    "abusive women",
    "fuck you too",
    "violence",
    "wit a lame nigga",
    "they all look",
    "alllivesmatter",
    "shithole countries",
    "fucking hate",
    "trailer trash",
    "kill all",
    "terrorist threat",
    "harassment",
    "kill yourself",
    "shitskin",
    "okay to be white",
    "fucking hate you"
  ],
  "white": [
    "full of white",
    "white trash",
    "white devil",
    "white",
    "are all white",
    "white boy",
    "white ass",
    "white bitch",
    "hillbilly",
    "whigger",
    "white christian",
    "white person",
    "all white",
    "white nigger",
    "redneck",
    "white honky",
    "wigger",
    "them white"
  ],
  "economy": [
    "ghetto"
  ],
  "immigrant": [
    "illegal immigrants",
    "immigrant not welcome",
    "immigrant terror",
    "mexcrement",
    "go back to where you come from",
    "muslim refugee",
    "illegal aliens",
    "refugee",
    "protect from immigrants",
    "negro",
    "refugees",
    "immigrant",
    "refugee invasion",
    "go back to where they come from",
    "refugees impact",
    "bring ebola",
    "immigrants",
    "illegal alien",
    "immigrant invasion",
    "bring disease"
  ],
  "mental": [
    "retard",
    "mongoloid",
    "retarded"
  ],
  "asians": [
    "asians",
    "ching chong",
    "chinaman"
  ]
}

In [ ]:
def create_corpus_from_comments(comment_list):
    corpus = list()
    for comment in comment_list:
        corpus.append(' '.join(clean_text(comment['body'])))
    return ' '.join(corpus)

In [ ]:
def get_slur_prob(corpus):
    """"corpus must be generated using create_corpus_from_comments function"""
    token_count = len(corpus.split())
    slur_prob   = dict()

    for slur_group in SLUR_WORDS.keys(): 
        slur_prob[slur_group] = 0  
        
        for slur_words in SLUR_WORDS[slur_group]: 
            slur_prob[slur_group] += corpus.count(slur_words) * len(slur_words.split())
        
        slur_prob[slur_group] /= token_count
    
    return slur_prob

In [ ]:
def compare_slur_probs(slur_prob_1, slur_prob_2, label1, label2):
    """Comparing two slur probabilities by plotting them
    """
    x = list(SLUR_WORDS.keys())
    y1 = [] 
    y2 = [] 

    for slur_group in x:
        y1.append(slur_prob_1[slur_group])
        y2.append(slur_prob_2[slur_group])

    ticks = np.arange(len(x))
    width = 0.30

    fig, ax = plt.subplots()
    subplot1 = ax.bar(ticks - width / 2, y1, width, label=label1, tick_label=x)
    subplot2 = ax.bar(ticks + width / 2, y2, width, label=label2, tick_label=x)

    ax.set_ylabel('Probability')
    ax.set_title(f'Probability of being a slur word')
    ax.set_xticks(ticks)
    ax.set_xticklabels(x, rotation=45, ha='right')
    ax.legend()
    plt.show()

# Analysis

In [ ]:
categories_1 = ['religion']
categories_2 = ['politics2']

In [ ]:
migration_list = get_migrated_users(categories_1, categories_2, categories_1_origin=True, require_migration=True)

In [ ]:
len(migration_list)

In [ ]:
AA, AN, NA, NN = partition_migrated_users(migration_list, categories_1, categories_2)

In [ ]:
AAset = set(AA)
ANset = set(AN)
NAset = set(NA) 
NNset = set(NN)

In [ ]:
def partition_comments(user_subset): # user_subset can be AAset, ANset, ..., 
    categories_1_comments = list()
    categories_2_comments = list()

    for cat in categories_1:
        for comment in comments[cat]:
            if comment['author'] in user_subset: 
                categories_1_comments.append(deepcopy(comment))
    
    for cat in categories_2:
        for comment in comments[cat]:
            if comment['author'] in user_subset: 
                categories_2_comments.append(deepcopy(comment))

    return categories_1_comments, categories_2_comments

In [ ]:
def get_random_comments(comment_list, n=100):
    n = min(n, len(comment_list)) 
    samples = random.sample(comment_list, n) 
    texts = list()
    for comment in samples:
        texts.append(' '.join(clean_text(comment['body']))) 
    return texts

In [ ]:
cat1c, cat2c = partition_comments(NNset)

In [ ]:
print(f'{len(cat1c)}, {len(cat2c)}')

In [ ]:
corpus_1 = create_corpus_from_comments(cat1c)
corpus_2 = create_corpus_from_comments(cat2c)

In [ ]:
slur_prob_1 = get_slur_prob(corpus_1) 
slur_prob_2 = get_slur_prob(corpus_2)

In [ ]:
compare_slur_probs(slur_prob_1, slur_prob_2, str(categories_1), str(categories_2))

In [ ]:
random_comments_1 = get_random_comments(cat1c)
random_comments_2 = get_random_comments(cat2c)

In [ ]:
create_latex_file(random_comments_1)

In [ ]:
create_latex_file(random_comments_2)